In [1]:
import pandas as pd
import numpy as np

# Dvisiveness Score

In [2]:
art = pd.read_csv('full_articles.csv')
art.head()

,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,"男模的大鸟,避孕膜,杜达雄\nHome\nHome\nBlogs\nPopular Twee...",aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
1,"Meet Elizabeth347, 26 (United States, Aixàs) |...",bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,left-center,163,BET,294
2,Trajectory Path Planning of Cable Driven Paral...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
3,Adverse drug reaction | Asian Journal of Pharm...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
4,Artificial Light at Night (ALAN) Research Lite...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,left,3,Mitú,7


In [3]:
groups = pd.read_csv('Journalist-Groups.csv')
groups['username'] = groups['username'].map(lambda x: x.lower())
print(groups.shape)
groups.head()

(2087, 2)


,username,Catergory
0,agkootenay,Television Network News and Public Affairs
1,ambankstw,Digital
2,ariquier,Television Network News and Public Affairs
3,asimendinger,Newspapers
4,aaronmehta,Television Network News and Public Affairs


In [4]:
users = pd.read_csv('Twitter-Usernames.csv')
users = users[['Journalist', 'Publisher', 'submitted username']]
users = users.rename(columns={'submitted username': 'username'})
print(users.shape)
users.head()

(3595, 3)


,Journalist,Publisher,username
0,"Adragna, Anthony",Politico,anthonyadragna
1,"Aftab, Mirza",BNA News,mirzaaftab
2,"Akin, Stephanie",Roll Call,stephanieakin
3,"Alder, Madison",BNA News,madisonalder
4,"Aleem, Zeeshan",Vox Media,zeeshanaleem


In [5]:
users_group = pd.merge(users, groups, how='inner', on='username')
users_group = users_group.drop_duplicates()
print(users_group.shape)
users_group.head()

(784, 4)


,Journalist,Publisher,username,Catergory
0,"Adragna, Anthony",Politico,anthonyadragna,Digital
2,"Akin, Stephanie",Roll Call,stephanieakin,Newspapers
3,"Alder, Madison",BNA News,madisonalder,Television Network News and Public Affairs
4,"Alder, Madison",BNA News,madisonalder,Digital
5,"Aleem, Zeeshan",Vox Media,zeeshanaleem,Television Network News and Public Affairs


In [6]:
users_group['Full Name'] = users_group['Journalist'].map(lambda x: x.split(', ')[1].lower()) + ' ' + users_group['Journalist'].map(lambda x: x.split(', ')[0].lower())
users_group.head()

,Journalist,Publisher,username,Catergory,Full Name
0,"Adragna, Anthony",Politico,anthonyadragna,Digital,anthony adragna
2,"Akin, Stephanie",Roll Call,stephanieakin,Newspapers,stephanie akin
3,"Alder, Madison",BNA News,madisonalder,Television Network News and Public Affairs,madison alder
4,"Alder, Madison",BNA News,madisonalder,Digital,madison alder
5,"Aleem, Zeeshan",Vox Media,zeeshanaleem,Television Network News and Public Affairs,zeeshan aleem


In [7]:
def find_author(text):
    lower = text.lower()

    for author in list(users_group['Full Name']):
        if author in lower:
            return author
        else:
            return np.nan

In [8]:
art['Author'] = art['text'].apply(find_author)
art.head()

,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes,Author
0,"男模的大鸟,避孕膜,杜达雄\nHome\nHome\nBlogs\nPopular Twee...",aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN
1,"Meet Elizabeth347, 26 (United States, Aixàs) |...",bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,left-center,163,BET,294,NaN
2,Trajectory Path Planning of Cable Driven Paral...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN
3,Adverse drug reaction | Asian Journal of Pharm...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN
4,Artificial Light at Night (ALAN) Research Lite...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,left,3,Mitú,7,NaN


In [9]:
def convert_to_num(target):
    # This will not be how we determine Fake News in final score.  The article will flow through the model.
    if target == 'The Daily Show (humor)':
        return 'Fake'
    elif target == 'Babylon Bee (Humor)':
        return 'Fake'
    else:
        return 'Real'

In [10]:
art['News Type'] = art['name'].apply(convert_to_num)
art.head()

,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes,Author,News Type
0,"男模的大鸟,避孕膜,杜达雄\nHome\nHome\nBlogs\nPopular Twee...",aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN,Real
1,"Meet Elizabeth347, 26 (United States, Aixàs) |...",bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,left-center,163,BET,294,NaN,Real
2,Trajectory Path Planning of Cable Driven Paral...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN,Real
3,Adverse drug reaction | Asian Journal of Pharm...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,NaN,Real
4,Artificial Light at Night (ALAN) Research Lite...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,left,3,Mitú,7,NaN,Real


In [11]:
#art = art[art['Author'].notna()]
# This will not be how we determine Bias in final score.  The article will flow through the model.
art_dum = pd.get_dummies(art, columns=['bias', 'News Type'])
art_dum.head()

,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,disagree,name,total_votes,Author,bias_allsides,bias_center,bias_left,bias_left-center,bias_right,bias_right-center,News Type_Fake,News Type_Real
0,"男模的大鸟,避孕膜,杜达雄\nHome\nHome\nBlogs\nPopular Twee...",aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1
1,"Meet Elizabeth347, 26 (United States, Aixàs) |...",bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,163,BET,294,NaN,0,0,0,1,0,0,0,1
2,Trajectory Path Planning of Cable Driven Paral...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1
3,Adverse drug reaction | Asian Journal of Pharm...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1
4,Artificial Light at Night (ALAN) Research Lite...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,3,Mitú,7,NaN,0,0,1,0,0,0,0,1


In [12]:
art_dum['total_articles'] = art_dum['bias_left'] + art_dum['bias_left-center'] + art_dum['bias_center'] + art_dum['bias_right'] + art_dum['bias_right-center']
print(art_dum.shape)
art_dum.head()

(509387, 19)


,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,disagree,name,total_votes,Author,bias_allsides,bias_center,bias_left,bias_left-center,bias_right,bias_right-center,News Type_Fake,News Type_Real,total_articles
0,"男模的大鸟,避孕膜,杜达雄\nHome\nHome\nBlogs\nPopular Twee...",aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1,1
1,"Meet Elizabeth347, 26 (United States, Aixàs) |...",bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,163,BET,294,NaN,0,0,0,1,0,0,0,1,1
2,Trajectory Path Planning of Cable Driven Paral...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1,1
3,Adverse drug reaction | Asian Journal of Pharm...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,106,AJ+,369,NaN,0,0,1,0,0,0,0,1,1
4,Artificial Light at Night (ALAN) Research Lite...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,3,Mitú,7,NaN,0,0,1,0,0,0,0,1,1


In [13]:
art_dum = art_dum[['Author', 'News Type_Fake', 'bias_left', 'bias_left-center', 'bias_center', 'bias_right', 'bias_right-center', 'total_articles']]
art_score = art_dum.groupby(['Author'], as_index=False).sum()
print(art_score.shape)
art_score.head()

(1, 8)


,Author,News Type_Fake,bias_left,bias_left-center,bias_center,bias_right,bias_right-center,total_articles
0,anthony adragna,0,0,3,0,0,0,3


In [14]:
art_score['Extreme Divide'] = (art_score['bias_left'] + art_score['bias_right'])/art_score['total_articles']
art_score['Some Divide'] = (art_score['bias_left-center'] + art_score['bias_right-center'])/art_score['total_articles']
art_score['Dvisiveness Score'] = (art_score['Extreme Divide']*.75) + (art_score['Some Divide']*.25)
art_score['Fake News Score'] = (art_score['News Type_Fake']/art_score['total_articles'])
art_score.head()

,Author,News Type_Fake,bias_left,bias_left-center,bias_center,bias_right,bias_right-center,total_articles,Extreme Divide,Some Divide,Dvisiveness Score,Fake News Score
0,anthony adragna,0,0,3,0,0,0,3,0.0,1.0,0.25,0.0


In [15]:
art_score.to_csv("Dvisiveness Score.csv", index=False)

# Social Score

In [16]:
tweets = pd.read_csv('User-Tweets-Category.csv')
print(tweets.shape)
tweets.head()

(14647, 3)


,username,tweet_text,Catergory
0,ABC7Brad,Certainly plenty of bad news in today’s Maryla...,Local TV Stations
1,ABC7Brad,@brucejohnson9 @wusa9 @SEDCScoop @PGCountyScoo...,Local TV Stations
2,ABC7Brad,This is just a tiny fraction of the line on rt...,Local TV Stations
3,ABC7Brad,"I first met @brucejohnson9 when ,many years ag...",Local TV Stations
4,ABC7Brad,RT @ABC7News: JUST IN via AP: U.S. clears seco...,Local TV Stations


In [17]:
def find_tweeter(text):
    lower = text.lower()

    for twitter in list(users_group['username']):
        if twitter in lower:
            return twitter
        else:   
            return np.nan

In [18]:
tweets['mentioned'] = tweets['tweet_text'].apply(find_tweeter)
tweets.head()

,username,tweet_text,Catergory,mentioned
0,ABC7Brad,Certainly plenty of bad news in today’s Maryla...,Local TV Stations,NaN
1,ABC7Brad,@brucejohnson9 @wusa9 @SEDCScoop @PGCountyScoo...,Local TV Stations,NaN
2,ABC7Brad,This is just a tiny fraction of the line on rt...,Local TV Stations,NaN
3,ABC7Brad,"I first met @brucejohnson9 when ,many years ag...",Local TV Stations,NaN
4,ABC7Brad,RT @ABC7News: JUST IN via AP: U.S. clears seco...,Local TV Stations,NaN


In [19]:
groups = groups.rename(columns={'Catergory': 'mentioned Category'})
tweets_cats = pd.merge(tweets, groups, how='left', left_on='mentioned', right_on='username')
tweets_cats.head()

,username_x,tweet_text,Catergory,mentioned,username_y,mentioned Category
0,ABC7Brad,Certainly plenty of bad news in today’s Maryla...,Local TV Stations,NaN,NaN,NaN
1,ABC7Brad,@brucejohnson9 @wusa9 @SEDCScoop @PGCountyScoo...,Local TV Stations,NaN,NaN,NaN
2,ABC7Brad,This is just a tiny fraction of the line on rt...,Local TV Stations,NaN,NaN,NaN
3,ABC7Brad,"I first met @brucejohnson9 when ,many years ag...",Local TV Stations,NaN,NaN,NaN
4,ABC7Brad,RT @ABC7News: JUST IN via AP: U.S. clears seco...,Local TV Stations,NaN,NaN,NaN


In [20]:
def matching(user, mentioned):
    if user == mentioned:
        return 1
    else:
        return 0

In [21]:
tweets_cats['matched'] = tweets_cats['Catergory'].apply(matching, args=['mentioned Category'])
tweets_cats['count'] = 1
tweets_cats.head()

,username_x,tweet_text,Catergory,mentioned,username_y,mentioned Category,matched,count
0,ABC7Brad,Certainly plenty of bad news in today’s Maryla...,Local TV Stations,NaN,NaN,NaN,0,1
1,ABC7Brad,@brucejohnson9 @wusa9 @SEDCScoop @PGCountyScoo...,Local TV Stations,NaN,NaN,NaN,0,1
2,ABC7Brad,This is just a tiny fraction of the line on rt...,Local TV Stations,NaN,NaN,NaN,0,1
3,ABC7Brad,"I first met @brucejohnson9 when ,many years ag...",Local TV Stations,NaN,NaN,NaN,0,1
4,ABC7Brad,RT @ABC7News: JUST IN via AP: U.S. clears seco...,Local TV Stations,NaN,NaN,NaN,0,1


In [22]:
tweets_cats = tweets_cats.rename(columns={'username_x': 'username'})
tweets_cats = tweets_cats[['username', 'matched', 'count']]
tweets_cats_score = tweets_cats.groupby(['username'], as_index=False).sum()
print(tweets_cats_score.shape)
tweets_cats_score.head()

(1873, 3)


,username,matched,count
0,ABC7Brad,0,10
1,ABC7Horace,0,2
2,ABC7Jay,0,5
3,ABC7Kevin,0,7
4,ABC7Sam,0,5


In [23]:
tweets_cats_score['Social Score'] = 1-(tweets_cats_score['matched']/tweets_cats_score['count'])
tweets_cats_score.head()

,username,matched,count,Social Score
0,ABC7Brad,0,10,1.0
1,ABC7Horace,0,2,1.0
2,ABC7Jay,0,5,1.0
3,ABC7Kevin,0,7,1.0
4,ABC7Sam,0,5,1.0


In [24]:
tweets_cats_score.to_csv("Social Score.csv", index=False)

# Overall Score

In [25]:
users_group['Journalist'] = users_group['Journalist'].map(lambda x: x.split(', ')[0]) + ' ' + users_group['Journalist'].map(lambda x: x.split(', ')[1])
users_group.head()

,Journalist,Publisher,username,Catergory,Full Name
0,Adragna Anthony,Politico,anthonyadragna,Digital,anthony adragna
2,Akin Stephanie,Roll Call,stephanieakin,Newspapers,stephanie akin
3,Alder Madison,BNA News,madisonalder,Television Network News and Public Affairs,madison alder
4,Alder Madison,BNA News,madisonalder,Digital,madison alder
5,Aleem Zeeshan,Vox Media,zeeshanaleem,Television Network News and Public Affairs,zeeshan aleem


In [30]:
div_users_group = pd.merge(users_group, art_score, how='left', left_on='Full Name', right_on='Author')
overall_score = pd.merge(div_users_group, tweets_cats_score, how='left', on='username')
print(overall_score.shape)
overall_score.head()

(784, 20)


,Journalist,Publisher,username,Catergory,Full Name,Author,News Type_Fake,bias_left,bias_left-center,bias_center,bias_right,bias_right-center,total_articles,Extreme Divide,Some Divide,Dvisiveness Score,Fake News Score,matched,count,Social Score
0,Adragna Anthony,Politico,anthonyadragna,Digital,anthony adragna,anthony adragna,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.25,0.0,0,12,1.0
1,Akin Stephanie,Roll Call,stephanieakin,Newspapers,stephanie akin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5,1.0
2,Alder Madison,BNA News,madisonalder,Television Network News and Public Affairs,madison alder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,1.0
3,Alder Madison,BNA News,madisonalder,Digital,madison alder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,1.0
4,Aleem Zeeshan,Vox Media,zeeshanaleem,Television Network News and Public Affairs,zeeshan aleem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,1.0


In [31]:
overall_score['Overall Score'] = ((1-overall_score['Dvisiveness Score'])*.45) + ((1-overall_score['Fake News Score'])*.30) + (overall_score['Social Score']*.25)
overall_score = overall_score[['Journalist', 'Publisher', 'username', 'Catergory', 
                               'Extreme Divide', 'Some Divide', 'Dvisiveness Score', 'Fake News Score', 'Social Score', 'Overall Score']]
overall_score.head()

,Journalist,Publisher,username,Catergory,Extreme Divide,Some Divide,Dvisiveness Score,Fake News Score,Social Score,Overall Score
0,Adragna Anthony,Politico,anthonyadragna,Digital,0.0,1.0,0.25,0.0,1.0,0.8875
1,Akin Stephanie,Roll Call,stephanieakin,Newspapers,NaN,NaN,NaN,NaN,1.0,NaN
2,Alder Madison,BNA News,madisonalder,Television Network News and Public Affairs,NaN,NaN,NaN,NaN,1.0,NaN
3,Alder Madison,BNA News,madisonalder,Digital,NaN,NaN,NaN,NaN,1.0,NaN
4,Aleem Zeeshan,Vox Media,zeeshanaleem,Television Network News and Public Affairs,NaN,NaN,NaN,NaN,1.0,NaN


In [32]:
overall_score.to_csv("Overall Score.csv",  index=False)